# Imports

In [ ]:
import pandas
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_file = 'data/continuous_dataset.csv'
df = pd.read_csv(data_file)

data_file = 'data/weekly_pre_dispatch_forecast.csv'
df_forecast_pre_dispatch = pd.read_csv(data_file)

In [ ]:
# This dataset contains the feature variables and dependent variable datetime.
df

In [ ]:
# Load forecast in the pre dispatch reports is the prediction made by the grid operator.
# This is not a feature for our model but could be compared with our predictions as an exploratory activity.
df_forecast_pre_dispatch

In [ ]:
df.describe()

In [ ]:
# Checking if there are missing values. None found.
df.isna().sum()

In [ ]:
# Making columns lower case for better readability.
df.columns = df.columns.str.lower()
df

In [ ]:
# Check the data types of the variables
df.dtypes

In [ ]:
# Checking if variable `school` has non-boolean values
df.school.value_counts()

In [ ]:
# Checking if variable `holiday` has non-boolean values
df.holiday.value_counts()

Checking if 0's in `holiday_id` matches the number of holidays based on `holiday`.

In [ ]:
assert df.holiday_id.value_counts()[0] == df.holiday.value_counts()[0]

In [ ]:
# datetime is a string. Splitting it to multiple columns will make plotting easier.
# Therefore, creating a new variable called `dt` of type pd.datetime by converting the values from `df.datetime`.
df['dt'] = pd.to_datetime(df.datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Checking if there are datetime conversion errors.
assert df.dt.isnull().sum() == 0

# delete datetime from the dataframe as dt supersedes it now.
del df['datetime']

In [ ]:
df

In [ ]:
df['dt_year'] = df['dt'].dt.year
df['dt_month'] = df['dt'].dt.month
df['dt_day'] = df['dt'].dt.day
df['dt_hour'] = df['dt'].dt.hour

# No need to separate minute and second values as they are always 0. Verified and confirmed.
# df['dt_minute'] = df.datetime.dt.minute
# df['dt_second'] = df.datetime.dt.second


In [ ]:
# Visually check all dt variables that they've been split correctly from a semantic viewpoint.
# Programmatic check was done above by checking for conversion errors.
df[df.columns[df.columns.str.match('^dt.*')]][::100]

In [ ]:
df.groupby(by=['dt_year', 'dt_month']).size()

In [ ]:
# df[['t2m_toc', 't2m_san', 't2m_dav', 'dt_year', 'dt_month']].groupby(by=['dt_year', 'dt_month'], group_keys=True).max().plot().bar()

In [ ]:
# distribution of the target variable
# %matplotlib inline
# plt.figure(figsize=(10,10))
# sns.histplot(df.nat_demand)
# sns.histplot(y_pred, color='red', bins=50, alpha=0.5)

df.dtypes

In [ ]:
df.nat_demand

In [ ]:
sns.histplot(df.nat_demand, color='red', bins=50, alpha=0.5)

In [ ]:
# See the distribution of the log1p version of the target variable
sns.histplot(np.log1p(df.nat_demand), color='red', bins=50, alpha=0.5)

In [ ]:
# Focusing on the long tail on the left. They seem to be outliers.
plt.xlim(0, 1000)
plt.ylim(0, 10)
sns.histplot(df.nat_demand, color='red', bins=50, alpha=0.5)

In [ ]:
# Looks like there are 8 very low demands. This is only a 0.017% of the total records.
# Without these outliers the national demand values are 'normally' distributed.
# Therefore, no need to log1p() the values.
round(df.nat_demand[df.nat_demand < 600].size / len(df.nat_demand) * 100, 3)

In [ ]:
sns.histplot(df.nat_demand[df.nat_demand > 600], color='red', bins=50, alpha=0.5)

In [ ]:
# Splitting the dataset to 80%, 20%, 20% for training, validation, and testing, respectively.
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
print(f'train:val:test split is {(len(df_train), len(df_val), len(df_test))}')

# Also create a copy of the complete dataframe with order intact to plot actual vs forecast based on the final model.
df_full = df.copy()
print(f'complete dataset is {len(df_full)}')

In [ ]:
# Resetting indices
df_full_train.reset_index(inplace=True)
df_train.reset_index(inplace=True)
df_val.reset_index(inplace=True)
df_test.reset_index(inplace=True)
df_full.reset_index(
    inplace=True)  # not necessary, but done so the for-loop below doesn't fail due to not having an index column.

In [ ]:
y_full_train = df_full_train.nat_demand
y_train = df_train.nat_demand
y_val = df_val.nat_demand
y_test = df_test.nat_demand
y_full = df_full.nat_demand

# log1p
# y_train = np.log1p(df_train.nat_demand)
# y_val = np.log1p(df_val.nat_demand)
# y_test = np.log1p(df_test.nat_demand)

In [ ]:
df_full

In [ ]:
# Removing unwanted variables
for c in ['nat_demand', 'dt', 'index']:
    del df_full_train[c]
    del df_train[c]
    del df_val[c]
    del df_test[c]
    del df_full[c]

In [ ]:
# Vectorize the features
dv = DictVectorizer(sparse=False)
train_dicts = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)

test_dicts = df_test.to_dict(orient='records')
X_test = dv.transform(test_dicts)

# Random Forest

Training a Random Forest Regressor by tuning 3 of its parameters, viz. n_estimators, max_depth and min_samples_leaf, to derive the best (lowest) RMSE score. The resulting RMSE will be set as the baseline. Once the baseline is set, an XGBoost Regressor will be trained and evaluated to see if we can achieve a model with a better RMSE.

### Benchmark 1
Train a model and measure performance with default `n_estimators`, `max_depth` and `
` values.

In [ ]:
rf = RandomForestRegressor(n_estimators=100,
                           max_depth=None,
                           min_samples_leaf=1,
                           random_state=1,
                           n_jobs=-1)
model = rf.fit(X_train, y_train)
y_val_pred = rf.predict(X_val)


#### Feature importance (Gini)

In [ ]:
ft_imp = list(zip(rf.feature_importances_, dv.get_feature_names_out()))
df_ft_imp = pd.DataFrame(ft_imp, columns=['score', 'feature']).sort_values(by='score', ascending=True)
plt.barh(df_ft_imp.feature, df_ft_imp.score)

#### Performance

In [ ]:
rf_performance = [('rmse', np.sqrt(mean_squared_error(y_val, y_val_pred))),
                  ('mae', mean_absolute_error(y_val, y_val_pred))]
pd.DataFrame(rf_performance, columns=['metric', 'score'])

### Benchmark 2
Training the model with tuned parameters.

#### Feature importance (Gini)

#### Tune `n_estimators` and `max_depth`

In [ ]:
# Finding the optimal max_depth and n_estimators
scores = []
for d in tqdm([20, 25, 30, 35, 40, 45, 50]):
    for n in tqdm(range(10, 201, 20)):
        rf = RandomForestRegressor(n_estimators=n,
                                   max_depth=d,
                                   random_state=1,
                                   n_jobs=-1)
        rf.fit(X_train, y_train)
        y_val_pred = rf.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        mae = mean_absolute_error(y_val, y_val_pred)
        scores.append((n, d, rmse, mae))

In [ ]:
df_scores = pd.DataFrame(scores, columns=['n_estimators', 'max_depth', 'rmse', 'mae'])
df_scores

In [ ]:
# plt.figure(figsize=(8, 6))
plt.xlabel('No. of estimators')
plt.ylabel('RMSE')
for d in tqdm([20, 25, 30, 35, 40, 45, 50]):
    plt.plot(df_scores[df_scores.max_depth == d].n_estimators,
             df_scores[df_scores.max_depth == d].rmse,
             label=f'max_depth={d}')
plt.legend()

#### Tune `min_samples_leaf`

In [ ]:
# Based on the above graph, optimal max_depth is 30.
max_depth = 30

# Finding the optimal min_samples_leaf
scores = []
for s in tqdm([1, 3, 5, 10, 50]):
    for n in tqdm(range(10, 201, 20)):
        rf = RandomForestRegressor(n_estimators=n,
                                   max_depth=max_depth,
                                   min_samples_leaf=s,
                                   random_state=1,
                                   n_jobs=-1)
        rf.fit(X_train, y_train)
        y_val_pred = rf.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        mae = mean_absolute_error(y_val, y_val_pred)
        scores.append((n, s, rmse, mae))

In [ ]:
df_scores = pd.DataFrame(scores, columns=['n_estimators', 'min_samples_leaf', 'rmse', 'mae'])
df_scores

In [ ]:
# plt.figure(figsize=(10, 8))
plt.xlabel('No. of estimators')
plt.ylabel('RMSE')
for s in [1, 3, 5, 10, 50]:
    plt.plot(df_scores[df_scores.min_samples_leaf == s].n_estimators,
             df_scores[df_scores.min_samples_leaf == s].rmse,
             label=f'min_samples_leaf={s}')
plt.legend()

In [ ]:
n_estimators = 150
max_depth = 30
min_samples_leaf = 1

rf = RandomForestRegressor(n_estimators=n_estimators,
                           max_depth=max_depth,
                           min_samples_leaf=min_samples_leaf,
                           random_state=1,
                           n_jobs=-1)
rf.fit(X_train, y_train)
y_val_pred = rf.predict(X_val)

In [ ]:
ft_imp = list(zip(rf.feature_importances_, dv.get_feature_names_out()))
df_ft_imp = pd.DataFrame(ft_imp, columns=['score', 'feature']).sort_values(by='score', ascending=True)
plt.barh(df_ft_imp.feature, df_ft_imp.score)

#### Performance

In [ ]:
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae = mean_absolute_error(y_val, y_val_pred)

rf_performance = [('rmse', rmse),
                  ('mae', mae)]
pd.DataFrame(rf_performance, columns=['metric', 'score'])

**Result:**
Benchmark 2 results is slightly better than Benchmark 1. Therefore, Benchmark 2 will be used as the benchmark for the model performance and will be used for comparison when measuring performance of the gradient boosting models in the next section.

# XGBoost

### Tune `eta`

In [ ]:
features = dv.get_feature_names_out()
dm_train = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dm_val = xgb.DMatrix(X_val, label=y_val, feature_names=features)
dm_test = xgb.DMatrix(X_test, label=y_test, feature_names=features)

In [ ]:
def train_gb_model(dm_train,
                   eta=0.3,
                   max_depth=6,
                   min_child_weight=1,
                   num_boost_round=201,
                   watchlist=[(dm_train, 'train'), (dm_val, 'val')]):
    xgb_params = {
        'eta': eta,
        'max_depth': max_depth,
        'min_child_weight': min_child_weight,

        'eval_metric': 'rmse',
        'objective': 'reg:squarederror',
        'nthread': -1,

        'seed': 1,
        'verbosity': 1
    }
    evals_result = {}
    model = xgb.train(params=xgb_params,
                      dtrain=dm_train,
                      num_boost_round=num_boost_round,
                      evals=watchlist,
                      evals_result=evals_result,
                      verbose_eval=False)

    columns = ['eta', 'iter', 'train_rmse', 'val_rmse']
    train_rmse_scores = list(evals_result['train'].values())[0] if watchlist is not None else []
    val_rmse_scores = list(evals_result['val'].values())[0] if watchlist is not None else []

    df_scores = pd.DataFrame(
        list(zip([eta] * len(train_rmse_scores),
                 range(1, len(train_rmse_scores) + 1),
                 train_rmse_scores,
                 val_rmse_scores
                 )), columns=columns)
    return model, df_scores

In [ ]:
scores = pd.DataFrame()
for eta in tqdm([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 1.0]):
    key = f'eta={eta}'
    _, df_scores = train_gb_model(dm_train,
                                  eta=eta,
                                  num_boost_round=201)
    scores = pd.concat([scores, df_scores])

In [ ]:
scores.sort_values(by='val_rmse', ascending=True).reset_index().iloc[::200]

In [ ]:
fig, axs = plt.subplots(1, 2)

fig.set_figwidth(20)

axs[0].set_title('Learning Rate - RMSE')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('RMSE (validation dataset)')
gs = scores.groupby('eta')
gs.get_group(1.00)
gs.groups.values()
for eta in gs.groups.keys():
    df = gs.get_group(eta)
    axs[0].plot(df.iter, df.val_rmse, label=f'eta={eta}')
    axs[0].legend()

axs[1].set_title('Learning Rate - RMSE (Zoomed)')
axs[1].set_xlabel('Iterations')
axs[1].set_ylabel('RMSE (validation dataset)')
axs[1].set_xlim([175, 200])
axs[1].set_ylim([75, 80])
gs = scores.groupby('eta')
gs.get_group(1.00)
gs.groups.values()
for eta in gs.groups.keys():
    df = gs.get_group(eta)
    axs[1].plot(df.iter, df.val_rmse, label=f'eta={eta}')
    axs[1].legend()

In [ ]:
# Base on the above analysis, eta=0.3 gives the best performance as the learning rate.
chosen_eta = 0.3

### Tune `max_depth`

In [ ]:
scores = {}
for max_depth in [3, 4, 6, 10, 14, 18]:
    key = f'max_depth={max_depth}'
    _, scores[key] = train_gb_model(dm_train,
                                    eta=chosen_eta,
                                    max_depth=max_depth,
                                    num_boost_round=201)

In [ ]:
fig, axs = plt.subplots(1, 2)

fig.set_figwidth(20)

axs[0].set_title('Max Depth - RMSE')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('RMSE (validation dataset)')
for key, df_scores in scores.items():
    axs[0].plot(df_scores.iter, df_scores.val_rmse, label=key)
    axs[0].legend()

axs[1].set_title('Max Depth - RMSE (Zoomed)')
axs[1].set_xlabel('Iterations')
axs[1].set_ylabel('RMSE (validation dataset)')
axs[1].set_xlim([175, 200])
axs[1].set_ylim([70, 100])
for key, df_scores in scores.items():
    axs[1].plot(df_scores.iter, df_scores.val_rmse, label=key)
    axs[1].legend()

In [ ]:
# The above analysis shows max_depth=10 gies the best performance.
chosen_max_depth = 10

### Tune `min_child_weight`

In [ ]:
scores = {}
for min_child_weight in [1, 10, 30, 40]:
    key = f'min_child_weight={min_child_weight}'
    _, scores[key] = train_gb_model(dm_train,
                                    eta=chosen_eta,
                                    max_depth=chosen_max_depth,
                                    min_child_weight=min_child_weight,
                                    num_boost_round=201)

In [ ]:
fig, axs = plt.subplots(1, 2)

fig.set_figwidth(20)

axs[0].set_title('Min Child Weight - RMSE')
axs[0].set_xlabel('Iterations')
axs[0].set_ylabel('RMSE (validation dataset)')
for min_child_weight, df in scores.items():
    df = scores[min_child_weight]
    axs[0].plot(df.iter, df.val_rmse, label=min_child_weight)
    axs[0].legend()

axs[1].set_title('Min Child Weight - RMSE (Zoomed)')
axs[1].set_xlabel('Iterations')
axs[1].set_ylabel('RMSE (validation dataset)')
axs[1].set_xlim([175, 200])
axs[1].set_ylim([70, 73])
for min_child_weight, df in scores.items():
    df = scores[min_child_weight]
    axs[1].plot(df.iter, df.val_rmse, label=min_child_weight)
    axs[1].legend()

In [ ]:
# The above analysis shows min_child_weight=30 gives the best performance.
chosen_min_child_weight = 30

### Final GB model

In [ ]:
# Training the model with train set and the chosen values for the parameters

(model, scores) = train_gb_model(dm_train=dm_train,
                                 eta=chosen_eta,
                                 max_depth=chosen_max_depth,
                                 min_child_weight=chosen_min_child_weight,
                                 num_boost_round=201)

In [ ]:
df_ft_imp = pd.DataFrame.from_dict(model.get_score(importance_type='weight'), orient='index').sort_values(by=0,
                                                                                                          ascending=True)
df_ft_imp.columns = ['score']
df_ft_imp

In [ ]:
plt.barh(df_ft_imp.index, df_ft_imp.score)

#### Performance

In [ ]:
gb_rmse = scores.sort_values(by='val_rmse').iloc[0, 3]
print(f'rmse of xgb model on test set = {gb_rmse}')

full_dicts = df_full.to_dict(orient='records')
X_full = dv.transform(full_dicts)
dm_full = xgb.DMatrix(X_full, label=y_full, feature_names=features)

y_full_pred = model.predict(dm_full)

plt.figure(figsize=(20, 8))
load_period = 24 * 14
actual = y_full[:load_period]
predict = y_full_pred[:load_period]
plt.plot(actual.index, list(actual), label='Actual Load')
plt.plot(actual.index, list(predict), color='red', label='Forecast')
plt.xlabel('Hours')
plt.ylabel('Load (MWh)')
plt.legend()

In [ ]:
# training with full_train and chosen params, and measure the performance
full_train_dicts = df_full_train.to_dict(orient='records')
X_full_train = dv.transform(full_train_dicts)
dm_full_train = xgb.DMatrix(X_full_train, label=y_full_train, feature_names=features)
(model, scores) = train_gb_model(dm_train=dm_full_train,
                                 eta=chosen_eta,
                                 max_depth=chosen_max_depth,
                                 min_child_weight=chosen_min_child_weight,
                                 num_boost_round=201,
                                 watchlist=[(dm_full_train, 'train'), (dm_test, 'val')])
gb_rmse = scores.sort_values(by='val_rmse').iloc[0, 3]
print(f'rmse of xgb model on full_train set = {gb_rmse}')

# We get much better RMSE compared to what we got from train set. Let's plot the full timeseries with predictions from the new model.
y_full_pred = model.predict(dm_full)

plt.figure(figsize=(20, 8))
load_period = 24 * 14
actual = y_full[:load_period]
predict = y_full_pred[:load_period]
plt.plot(actual.index, list(actual), label='Actual Load')
plt.plot(actual.index, list(predict), color='red', label='Forecast')
plt.xlabel('Hours')
plt.ylabel('Load (MWh)')
plt.legend()

# Export Model

In [ ]:
import bentoml

# retraining without the feature_names in the dmatrix, as otherwise predict() will fail later in the pipeline.
dm_full_train = xgb.DMatrix(X_full_train, label=y_full_train)
(model, scores) = train_gb_model(dm_train=dm_full_train,
                                 eta=chosen_eta,
                                 max_depth=chosen_max_depth,
                                 min_child_weight=chosen_min_child_weight,
                                 num_boost_round=201,
                                 watchlist=None)
bentoml.xgboost.save_model("load_forecast_model", model,
                           custom_objects={
                               "dictVectorizer": dv
                           },
                           signatures={
                               "predict": {
                                   "batchable": True,
                                   "batch_dim": 0
                               }
                           })

Test request payload from dict(df_test.iloc[0])

```json
{
  "t2m_toc": 25.6113220214844,
  "qv2m_toc": 0.01747758,
  "tql_toc": 0.043762207,
  "w2m_toc": 15.885400482402956,
  "t2m_san": 23.8613220214844,
  "qv2m_san": 0.016439982,
  "tql_san": 0.03894043,
  "w2m_san": 6.2321456709303815,
  "t2m_dav": 22.9472595214844,
  "qv2m_dav": 0.01531083,
  "tql_dav": 0.062301636,
  "w2m_dav": 3.6011136954933645,
  "holiday_id": 0.0,
  "holiday": 0.0,
  "school": 0.0,
  "dt_year": 2019.0,
  "dt_month": 1.0,
  "dt_day": 8.0,
  "dt_hour": 20.0
}
```

In [ ]:
list(df_full.columns)

In [66]:
cols = ['nat_demand',
        't2m_toc',
        'qv2m_toc',
        'tql_toc',
        'w2m_toc',
        't2m_san',
        'qv2m_san',
        'tql_san',
        'w2m_san',
        't2m_dav',
        'qv2m_dav',
        'tql_dav',
        'w2m_dav',
        'holiday_id',
        'holiday',
        'school',
        'dt',
        'dt_year',
        'dt_month',
        'dt_day',
        'dt_hour']
json_full = df_full[cols][:500].to_json(orient='records')

In [67]:
import json
parsed_json = json.loads(json_full)

In [68]:
with open("assets/output.json", 'w') as f_out:
  json.dump(parsed_json, f_out)

In [ ]:
json_full